добавьтеПрежде чем начать добавьте файлы model_lstm.h5 
(https://drive.google.com/file/d/1wj1TNoRcMaq9rmwnQS2k0ONbQZVxO3v_/view?usp=sharing) и tokenizer_twit.pkl 
(https://drive.google.com/file/d/1--J9hC2vlaZJcGHURCwxz5Pq4zMPUgBb/view?usp=sharing) на гугл диск

# Import libraries

In [ ]:
import pandas as pd
import re
import numpy as np

import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import tensorflow as tf 
import pickle

Using TensorFlow backend.


In [ ]:
# from Google Colab
from keras.models import load_model
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# install pymystem3 library for google colab
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /bin

--2020-07-31 04:53:24--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.241, 5.45.205.243, 5.45.205.242, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.241|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskm904.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2020-07-31 04:53:24--  http://cache-mskm904.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskm904.cdn.yandex.net (cache-mskm904.cdn.yandex.net)... 5.45.220.14, 2a02:6b8:0:2002::15
Connecting to cache-mskm904.cdn.yandex.net (cache-mskm904.cdn.yandex.net)|5.45.220.14|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz.1’

mystem-3.0-linux3.1 100%[===================>] 

# Load model

In [ ]:
# enter your folders
folder_for_model = 'your_folder'
folder_for_tokenizer = 'your_folder'

# load model
name_model = '/model_lstm.h5'
PATH = '/content/drive/My Drive/' + folder_for_model + name_model
model = tf.keras.models.load_model(PATH)
print('Model loaded')

# load tokenizer
name_tokenizer = '/tokenizer_twit.pkl'
PATH = '/content/drive/My Drive/' + folder_for_tokenizer + name_tokenizer
with open(PATH, 'rb') as file:
    tokenizer = pickle.load(file)
print('Tokenizer loaded')

Model loaded
Tokenizer loaded


# Predict

In [ ]:
nltk.download("stopwords")
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^А-Яа-я0-9]+"

def preprocess_text(text):
    # Remove link,user and special characters
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in russian_stopwords:
          tokens.append(token)

    text = " ".join(tokens)

    # lematize text
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
SENTIMENT_THRESHOLDS = [0.4, 0.7]
SEQUENCE_LENGTH = 300

def decode_sentiment(score, include_neutral=True):
    if include_neutral:        
        label = 'NEUTRAL'
        if score <= SENTIMENT_THRESHOLDS[0]:
            label = 'NEGATIVE'
        elif score >= SENTIMENT_THRESHOLDS[1]:
            label = 'POSITIVE'

        return label
    else:
        return 'NEGATIVE' if score < 0.5 else 'POSITIVE'

def predict(text, include_neutral=True):

    text = preprocess_text(text)
    
    x_test = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=SEQUENCE_LENGTH)

    score = model.predict([x_test])[0][0]

    label = decode_sentiment(score, include_neutral=include_neutral)

    return {"label": label, "score": float(score)} 

# Example

In [ ]:
predict('я люблю слушать музыку')

{'label': 'POSITIVE', 'score': 0.9680542945861816}

In [ ]:
predict('я не стану смотреть этот скучный фильм')

{'label': 'NEGATIVE', 'score': 0.2484307587146759}

In [ ]:
predict('мы были в этом ресторане', include_neutral=True)

{'label': 'NEUTRAL', 'score': 0.6624805331230164}

In [ ]:
predict('мы были в этом ресторане', include_neutral=False)

{'label': 'POSITIVE', 'score': 0.6624805331230164}

Your text

In [ ]:
text = 'enter your text'
predict(text)